## GPR & LiDAR Figure Map(s)


In [1]:
import sklearn.metrics as stat
import pandas as pd
import numpy as np

import shapely.geometry
from geoalchemy2.shape import from_shape
import geoalchemy2.functions as gfunc
import matplotlib.pyplot as plt
import datetime

%config InlineBackend.figure_format='retina'



In [2]:
#import data - initial GPR lines (showing TWT)
GPR1 = pd.read_csv('GPRLiDAR1.csv')
GPR2 = pd.read_csv('GPRLiDAR2.csv')

,test,twt,easting,northing,lidarSD,date,v,e,dens,SWE
0,1,8.950,743120.221029,4.323490e+06,1.0,2020-01-28 12:00:00.000000000,0.223464,1.802306,405.325444,0.405325
1,2,8.000,742494.300029,4.324476e+06,1.0,2020-01-29 04:48:00.000000000,0.250000,1.440000,236.686391,0.236686
2,3,7.700,742527.560029,4.322792e+06,1.0,2020-01-29 00:00:00.000000000,0.259740,1.334025,183.431953,0.183432
3,4,8.735,743360.034029,4.324253e+06,1.0,2020-01-29 12:00:00.000000000,0.228964,1.716755,367.159763,0.367160
4,5,9.325,744352.974030,4.322606e+06,1.0,2020-01-30 00:00:00.000000000,0.214477,1.956502,471.893491,0.471893


In [5]:
ax = plt.plot(GPR1['easting'],GPR1['northing'])
plt.show()

OverflowError: Exceeded cell block limit in Agg.  Please set the value of rcParams['agg.path.chunksize'], (currently 0) to be greater than 100 or increase the path simplification threshold(rcParams['path.simplify_threshold'] = 0.111111111111 by default and path.simplify_threshold = 0.111111111111 on the input).

<Figure size 432x288 with 1 Axes>

In [ ]:
ax = GPR1.plot(column='twt', legend=True, cmap='PuBu')
# Use non-scientific notation for x and y ticks
ax.ticklabel_format(style='plain', useOffset=False)
# Set the various plots x/y labels and title.
ax.set_title('Grand Mesa Jan 28th - Feb 1' )
ax.set_xlabel('Easting [m]')
ax.set_ylabel('Northing [m]')

In [ ]:
ax = GPR2.plot(column='twt', legend=True, cmap='PuBu')
# Use non-scientific notation for x and y ticks
ax.ticklabel_format(style='plain', useOffset=False)
# Set the various plots x/y labels and title.
ax.set_title('Grand Mesa Jan 28th - Feb 1' )
ax.set_xlabel('Easting [m]')
ax.set_ylabel('Northing [m]')

In [ ]:
# import data - GPRLiDAR match data 
GPRLiDAR1 = pd.read_csv('GPRLiDAR1.csv')
GPRLiDAR2 = pd.read_csv('GPRLiDAR2.csv')

In [ ]:
## showing resampled to the Lidar grid points
#plot to see bounding box
# Get the Matplotlib Axes object from the dataframe object, color points by density value
ax = GPRLiDAR1.plot(column='dens', legend=True, cmap='PuBu')
# Use non-scientific notation for x and y ticks
ax.ticklabel_format(style='plain', useOffset=False)
# Set the various plots x/y labels and title.
ax.set_title('Grand Mesa Jan 28th - Feb 1' )
ax.set_xlabel('Easting [m]')
ax.set_ylabel('Northing [m]')


ax = GPRLiDAR2.plot(column='dens', legend=True, cmap='PuBu')
# Use non-scientific notation for x and y ticks
ax.ticklabel_format(style='plain', useOffset=False)
# Set the various plots x/y labels and title.
ax.set_title('Grand Mesa Feb. 4 - 9th 2020' )
ax.set_xlabel('Easting [m]')
ax.set_ylabel('Northing [m]')


# GPR-LiDAR basic statistics

In [ ]:
GPRLiDAR1.describe().transpose()